## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [4]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-afrimcamp/02-tracking-experiment/running-mlflow-examples/artifacts_local/0', creation_time=1719082080859, experiment_id='0', last_update_time=1719082080859, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("remi-experiment-2")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/06/22 18:50:11 INFO mlflow.tracking.fluent: Experiment with name 'remi-experiment-2' does not exist. Creating a new experiment.


default artifacts URI: '/workspaces/mlops-afrimcamp/02-tracking-experiment/running-mlflow-examples/artifacts_local/1/1f71f7e03abd47d0ada3b006d0cc59d3/artifacts'


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-afrimcamp/02-tracking-experiment/running-mlflow-examples/artifacts_local/1', creation_time=1719082211462, experiment_id='1', last_update_time=1719082211462, lifecycle_stage='active', name='remi-experiment-2', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-afrimcamp/02-tracking-experiment/running-mlflow-examples/artifacts_local/0', creation_time=1719082080859, experiment_id='0', last_update_time=1719082080859, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [9]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [10]:
client.search_registered_models()

[]

In [23]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)


AttributeError: 'MlflowClient' object has no attribute 'list_run_infos'

In [18]:
run_id = client.search_experiments(filter_string="name='remi-experiment-2'").run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)



AttributeError: 'PagedList' object has no attribute 'run_id'

In [22]:
run_id = client.search_experiments(filter_string="name='remi-experiment-2'")[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

AttributeError: 'Experiment' object has no attribute 'run_id'